# Playlist analyzer

## Import the spotify api

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd

# Define your credentials and redirection URI
SPOTIPY_CLIENT_ID = 'dda606479dd244a9b0948c01f155ac7a'
SPOTIPY_CLIENT_SECRET = 'd714739739a04b0fa662e521e1a85ff8'
SPOTIPY_REDIRECT_URI = 'http://localhost:3000'

# Create the Spotify client

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET,
    redirect_uri=SPOTIPY_REDIRECT_URI,
    # scope=scope
))

playlists = spotify.user_playlists('liamkrenz')
name = input("Enter playlist name: ")

playlist_id = 0
for i in playlists["items"] : 
    if i["name"] == name :
        playlist_id = i["id"]
playlist_data = spotify.playlist(playlist_id)

## Extract song data on each song

In [2]:
playlist = []
for i in playlist_data["tracks"]["items"] : 
    playlist.append(spotify.audio_features(i["track"]["id"]))

print(playlist[1])


[{'danceability': 0.541, 'energy': 0.674, 'key': 1, 'loudness': -7.04, 'mode': 0, 'speechiness': 0.0841, 'acousticness': 0.00649, 'instrumentalness': 9.76e-05, 'liveness': 0.235, 'valence': 0.123, 'tempo': 103.139, 'type': 'audio_features', 'id': '0YsGMHid6sFq5PcToe3JZE', 'uri': 'spotify:track:0YsGMHid6sFq5PcToe3JZE', 'track_href': 'https://api.spotify.com/v1/tracks/0YsGMHid6sFq5PcToe3JZE', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0YsGMHid6sFq5PcToe3JZE', 'duration_ms': 317293, 'time_signature': 4}]


### Convert the new data into a dataframe including the name of each song

In [4]:
collumns = ["acousticness", "danceability", "energy", "instrumentalness", "liveness", "loudness", "speechiness", "tempo", "valence"]

df = pd.DataFrame(playlist, columns=collumns)

ValueError: 9 columns passed, passed data had 1 columns